# Worksheet 07

Name: SHOWNDARYA MADHAVAN 
UID: U10380918

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)


[3.8072423628854013, 6.398917522822156, 4.698397431813143, 3.3786991768109917, 2.928225559916981, 3.8750909456607396, 6.978272567435605, 5.838260281060971, 6.909724724083313, 3.2131845581151435]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[5.7056401866709585, 8.155270729530452, 7.556262626012456, 8.219751163914957, 9.542533901670765, 8.756356224563282, 7.98273528074294, 8.464532754221539, 8.049033394952934, 7.375064125088716]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 =  c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[3.2131845581151435, 6.909724724083313, 5.838260281060971, 7.375064125088716, 6.978272567435605, 8.049033394952934, 8.464532754221539, 7.98273528074294, 3.8750909456607396, 8.756356224563282]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.909724724083313, 5.838260281060971, 7.375064125088716, 6.978272567435605, 8.049033394952934, 8.464532754221539, 7.98273528074294, 8.756356224563282]
[3.2131845581151435, 3.8750909456607396]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 7.544247419018663,  mean_2 = 3.5441377518879413
var_1 = 0.8031468782352829,  var_2 = 16.110407365541064


The points are far from the true values

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]* prob_c[0]/prob_x )
    prob_c1_x.append( pdf_i[1]* prob_c[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  3.2131845581151435
probability of observing that point if it came from cluster 0 =  2.4066123273273237e-07
probability of observing that point if it came from cluster 1 =  0.02475779177560986
point =  6.909724724083313
probability of observing that point if it came from cluster 0 =  0.36356145728096995
probability of observing that point if it came from cluster 1 =  0.024228510610132092
point =  5.838260281060971
probability of observing that point if it came from cluster 0 =  0.05204317008268072
probability of observing that point if it came from cluster 1 =  0.024513215439253593
point =  7.375064125088716
probability of observing that point if it came from cluster 0 =  0.4858245732974099
probability of observing that point if it came from cluster 1 =  0.024072707721552084
point =  6.978272567435605
probability of observing that point if it came from cluster 0 =  0.38750772599750655
probability of observing that point if it came from cluster 1 =  0.024206764897269358
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7766869221054313,  P(C_2) = 0.22331307789456867
mean_1 = 7.559053606991862,  mean_2 = 3.910238548412796
var_1 = 0.8033661014375807,  var_2 = 16.22907934003326


The PC1 has increased but the PC2 has decreased. The mean for 1 has increaed by for 2 has decreased. The variances of 1 has increased but 2 has decreased.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0]* prob_c[0]/prob_x )
    prob_c1_x.append( pdf_i[1]* prob_c[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  3.2131845581151435
probability of observing that point if it came from cluster 0 =  2.1953728599869998e-07
probability of observing that point if it came from cluster 1 =  0.024559277881808516
point =  6.909724724083313
probability of observing that point if it came from cluster 0 =  0.3582099205622016
probability of observing that point if it came from cluster 1 =  0.0241656586105285
point =  5.838260281060971
probability of observing that point if it came from cluster 0 =  0.05008514513401117
probability of observing that point if it came from cluster 1 =  0.024409083061536073
point =  7.375064125088716
probability of observing that point if it came from cluster 0 =  0.48373424854113617
probability of observing that point if it came from cluster 1 =  0.024028053817048763
point =  6.978272567435605
probability of observing that point if it came from cluster 0 =  0.38239128097814723
probability of observing that point if it came from cluster 1 =  0.024146585810335082
point =  

Some points still remained in the same cluster, but some probablities changed so the clusters are reassigned

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
probabilities= zip(data, prob_c0_x, prob_c1_x)
for prob in probabilities:
    if prob[1]>prob[2]:
        print(str(prob[0])+" is in cluster 0")
    else:
        print(str(prob[0])+" is in cluster 1")

3.2131845581151435 is in cluster 1
6.909724724083313 is in cluster 0
5.838260281060971 is in cluster 0
7.375064125088716 is in cluster 0
6.978272567435605 is in cluster 0
8.049033394952934 is in cluster 0
8.464532754221539 is in cluster 0
7.98273528074294 is in cluster 0
3.8750909456607396 is in cluster 1
8.756356224563282 is in cluster 0


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.